# Install Libraries

In [1]:
!pip install assemblyai
!pip install python-dotenv
#!pip install openai==0.28
#!pip install --upgrade openai
!pip uninstall openai -y
!pip install openai
!pip install speechrecognition
!pip install git+https://github.com/openai/whisper.git
!pip install pyttsx3
!pip install pydub
!pip install ffprobe
!pip install ffmpeg
!pip install gradio

Found existing installation: openai 1.65.2
Uninstalling openai-1.65.2:
  Successfully uninstalled openai-1.65.2
  Using cached openai-1.65.2-py3-none-any.whl.metadata (27 kB)
Using cached openai-1.65.2-py3-none-any.whl (473 kB)
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-hrnwe_so
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-hrnwe_so
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Importing Libarries

In [2]:
import assemblyai as aai
import linecache
import os
import glob
import textwrap
import speech_recognition as sr
from transformers import pipeline
import pyttsx3
import openai
from openai import OpenAI
from dotenv import load_dotenv
import torch
import whisper
from transformers import pipeline
from pydub import AudioSegment
from pydub.utils import make_chunks
import gradio as gr
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"

Mounted at /content/drive


# API Key Setup

In [3]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env

In [4]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [39]:
# Load Whisper Model for Speech-to-Text
#whisper_model = whisper.load_model("base")

# Load Sentiment Analysis Model
#sentiment_analyzer = pipeline("sentiment-analysis")

# Audio to Text Transcribe

In [23]:
def transcribe_audio(audio_file):
    m4a_file = audio_file
    wav_filename = m4a_file.split(".")[0] + '.wav'

    sound = AudioSegment.from_file(m4a_file, format='m4a')
    file_handle = sound.export(wav_filename, format='wav')

    # Load the audio file
    audio = AudioSegment.from_file(wav_filename, format='wav')

    # Define chunk length in milliseconds (e.g., 30 seconds)
    chunk_length_ms = 30000

    # initialize the recognizer
    r = sr.Recognizer()

    # Split the audio into chunks
    chunks = make_chunks(audio, chunk_length_ms)
    text_list = []
    # Process each chunk individually using SpeechRecognition
    for i, chunk in enumerate(chunks):
        chunk_name = f"chunk{i}.wav"
        chunk.export(chunk_name, format="wav")

        # Now use SpeechRecognition on each chunk_name
        with sr.AudioFile(chunk_name) as source:
            # listen for the data (load audio to memory)
            audio_data = r.record(source)
            # recognize (convert from speech to text)
            try:  # Use try-except to handle potential errors
                text = r.recognize_google(audio_data)
                #print(text)
                text = text.replace('\n', ' ')
                text_list.append(text)
            except sr.UnknownValueError:
                print("Google Speech Recognition could not understand audio")
            except sr.RequestError as e:
                print(f"Could not request results from Google Speech Recognition service; {e}")

    text_excerpt = ' '.join(text_list)
    return text_excerpt

In [ ]:
def combine_wav_files(file_paths, output_path):
    """Combines multiple WAV files into one.

    Args:
        file_paths (list): List of paths to the WAV files to combine.
        output_path (str): Path to save the combined WAV file.
    """
    combined_audio = AudioSegment.empty()
    for file_path in file_paths:
        audio = AudioSegment.from_wav(file_path)
        combined_audio += audio
    combined_audio.export(output_path, format="wav")

#combine_wav_files(glob.glob('/content/chunk*.wav'), '/content/combined_audio.wav')

# Sentiment Analysis

In [31]:
def analyze_sentiment(text_excerpt):
    sentiment_messages = [
            {"role": "system", "content": "You are an AI language model trained to analyze and detect the sentiment of a conversation between customer and customer service agent."}
          ]
    sentiment_prompt = f"Analyze the text excerpt {text_excerpt} and determine the sentiment of the conversation in a single word: Positive, Negative or Neutral"
    sentiment_messages.append({"role": "user", "content": sentiment_prompt})
    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    )

    sentiment_analysis_chat_completion = client.chat.completions.create(
        messages=sentiment_messages,
        model="gpt-4o",
        temperature=0
    )
    return sentiment_analysis_chat_completion.choices[0].message.content

# Extract Reason

In [24]:
def extract_reason(text_excerpt):
    key_topic_messages = [
            {"role": "system", "content": "You are an AI language model trained to analyze a conversation between customer and customer service agent. You help finding out the reason of the customer call."}
          ]

    key_points_prompt = f"Analyze the text excerpt {text_excerpt} and determine the reason of the customer call in few words without any names mentioned."
    key_topic_messages.append({"role": "user", "content": key_points_prompt})
    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    )

    key_topic_chat_completion = client.chat.completions.create(
        messages=key_topic_messages,
        model="gpt-4o",
        temperature=0
    )
    #print(key_topic_chat_completion.choices[0].message.content)
    reason = key_topic_chat_completion.choices[0].message.content
    return reason

# Extract Action Plan

In [35]:
def extract_action_plan(text_excerpt, reason):
    action_plans_messages = [
                {"role": "system", "content": "You are an AI language model trained to analyze a conversation between customer and customer service agent. You help finding out the action plans to be taken with respect to the customer call reason."}
              ]

    action_plans_prompt = f"Analyze the text excerpt {text_excerpt} and {reason} and determine the concised action plans with respect to the customer call to resolve without mentioning any specific names."
    action_plans_messages.append({"role": "user", "content": action_plans_prompt})
    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    )

    action_plan_chat_completion = client.chat.completions.create(
        messages=action_plans_messages,
        model="gpt-4o",
        temperature=0
    )
    action_plan = action_plan_chat_completion.choices[0].message.content
    return action_plan

# Analyze Audio

In [36]:
def analyze_audio(audio_file):
    # Transcribe Audio
    text_excerpt = transcribe_audio(audio_file=audio_file)

    # Sentiment Analysis
    sentiment = analyze_sentiment(text_excerpt)

    # Reason Analysis
    reason = extract_reason(text_excerpt)

    # Action Plan
    action_plan = extract_action_plan(text_excerpt, reason)

    #return text_excerpt, sentiment, reason, action_plan
    return sentiment, reason, action_plan


In [38]:
#analyze_audio(audio_file='/content/call1.m4a')

# Customer Service Center Audio Call Analysis Application

In [37]:
# Gradio UI
demo = gr.Interface(
    fn=analyze_audio,
    inputs=gr.Audio(type="filepath", label="Upload .m4a Audio File"),
    outputs=[
        #gr.Textbox(label="Text Excerpt"),
        gr.Textbox(label="Sentiment"),
        gr.Textbox(label="Reason of Call"),
        gr.Textbox(label="Action Plan")
    ],
    title="Customer Service Center Audio Call Analysis",
    description="Upload a customer service call recording (.m4a) to analyze sentiment, reason, and action plans."
)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://df1ce99181576e2089.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
